In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from pathlib import Path

# Paths
RAW_PATH = Path("../data/raw/dataset.xlsx")
OUT_PATH = Path("../data/processed/01_cleaned_data.csv")
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

# Load
raw = pd.read_excel(RAW_PATH)
# Normalize column names
raw.columns = (raw.columns
               .str.replace(u"\xa0", " ", regex=False)
               .str.replace(r"\s+", " ", regex=True)
               .str.strip())

if "Patient ID" not in raw.columns:
    raise KeyError("'Patient ID' column is required.")

# Convert non-ID to numeric
non_id_cols = [c for c in raw.columns if c != "Patient ID"]
raw[non_id_cols] = raw[non_id_cols].apply(pd.to_numeric, errors="coerce")

# Deduplicate by patient ID
df = raw.drop_duplicates(subset=["Patient ID"]).copy()
feature_cols = [c for c in df.columns if c != "Patient ID"]

# Patient filter: >=10 non-missing labs
patient_counts = df[feature_cols].notna().sum(axis=1)
df = df.loc[patient_counts >= 10].copy()

# Feature filter: <80% missing
missing_frac = df[feature_cols].isna().mean()
kept_features = missing_frac[missing_frac < 0.80].index.tolist()
final_cols = ["Patient ID"] + kept_features

df = df[final_cols].copy()

# Impute + scale numeric features
numeric_cols = [c for c in df.columns if c != "Patient ID"]
imputer = SimpleImputer(strategy="median")
scaler = StandardScaler()

imputed = imputer.fit_transform(df[numeric_cols])
scaled = scaler.fit_transform(imputed)

processed = df.copy()
processed[numeric_cols] = scaled

print(f"Patients retained: {processed.shape[0]}")
print(f"Features retained: {len(numeric_cols)}")

processed.to_csv(OUT_PATH, index=False)
print(f"Saved cleaned data to {OUT_PATH}")



Patients retained: 603
Features retained: 40
Saved cleaned data to ..\data\processed\01_cleaned_data.csv
